In [ ]:
import pandas as pd
import numpy as np
import config
import covariance
import utils
import boto3
import clean_data

from importlib import reload

reload(covariance)
reload(config)
reload(utils)

In [ ]:
file_keys = utils.list_s3_files(config.BUCKET_NAME, "clean/")

for file_key in file_keys:
    df = utils.read_s3_file(file_key).set_index("date")
    df.index = pd.to_datetime(df.index)
    msfe_list = covariance.forecast_cov_matrices(df, config.DATES[0:2], config.L)
    msfe_df = pd.concat(msfe_list)
    print(f"Mean MSFE for {file_key}: {msfe_df.mean()}")
    original_filename = file_key.split("/")[-1]
    new_filename = original_filename.replace(".csv", "_forecast_error.csv")
    output_file_key = f"output/{new_filename}"
    utils.write_s3_file(msfe_df, output_file_key)

In [ ]:
# Parameters
dates = pd.date_range("19900101", "20240401", freq="MS")
L = 120

# Load data
df = utils.read_s3_file("clean/btm.csv").set_index("date")
df.index = pd.to_datetime(df.index)

# Forecast covariance matrices
msfe_list = covariance.forecast_cov_matrices(df, dates[0:5], L)

# # Combine results into a single DataFrame
msfe_df = pd.concat(msfe_list)
print(msfe_df)

# # Print the mean of MSFE values
print(msfe_df.mean())